# PyTechnicalIndicators: Full Walkthrough

This notebook will walk you through the end-to-end workflow for research with [PyTechnicalIndicators](https://github.com/chironmind/PyTechnicalIndicators):

- Fetching fresh OHLCV data from Binance
- Calculating multiple technical indicators
- Systematic evaluation of RSI model variants
- Visualizing price and indicators
- Ranking and comparing indicator performance

> **Requirements**: `pytechnicalindicators`, `pandas`, `requests`, `plotly`
> 
> Install with: 
> ```bash
> pip install pytechnicalindicators pandas requests plotly
> ```

---

## 1. Fetch Fresh Data from Binance

We will use [Binance's free API](https://binance-docs.github.io/apidocs/spot/en/#kline-candlestick-data) to get daily OHLCV data for a crypto symbol (e.g. `BTCUSDT`).

In [ ]:
import requests
import pandas as pd

def fetch_binance_ohlcv(symbol="BTCUSDT", interval="1d", limit=365):
    url = "https://api.binance.com/api/v3/klines"
    params = {"symbol": symbol, "interval": interval, "limit": limit}
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    df = pd.DataFrame(data, columns=[
        "OpenTime", "Open", "High", "Low", "Close", "Volume",
        "CloseTime", "QuoteAssetVolume", "NumTrades", "TakerBuyBase", "TakerBuyQuote", "Ignore"
    ])
    df["Date"] = pd.to_datetime(df["OpenTime"], unit="ms")
    cols = ["Date", "Open", "High", "Low", "Close", "Volume"]
    df = df[cols].astype({"Open": float, "High": float, "Low": float, "Close": float, "Volume": float})
    df = df.sort_values("Date").reset_index(drop=True)
    return df

# Fetch data
df = fetch_binance_ohlcv("BTCUSDT", "1d", 365)
df.tail()

## 2. Clean Data & Prepare Series

Prepare series as lists for indicator calculations.

In [ ]:
df = df.dropna(subset=["Open", "High", "Low", "Close"])
close = df["Close"].tolist()
high  = df["High"].tolist()
low   = df["Low"].tolist()
open_ = df["Open"].tolist()
volume = df["Volume"].tolist()

## 3. Calculate Technical Indicators

Let's calculate some basic indicators: SMA, ATR.

In [ ]:
import pytechnicalindicators as pti

# 20-period simple moving average
sma_20 = pti.moving_average.bulk.moving_average(close, period=20, moving_average_type="simple")
df.loc[df.index[-len(sma_20):], "SMA_20"] = sma_20

# 14-period exponential ATR
atr_14 = pti.other_indicators.bulk.average_true_range(close=close, high=high, low=low, constant_model_type="exponential", period=14)
df.loc[df.index[-len(atr_14):], "ATR_14"] = atr_14

df.tail()

## 4. Systematic RSI Variant Evaluation

Let's evaluate different `constant_model_type` variants for RSI with a fixed period and generate a simple next-day-up "signal score" as in the Rust tutorial.

In [ ]:
CONSTANT_MODELS = ["simple", "smoothed", "exponential", "median", "mode"]
RSI_PERIOD = 5
OVERSOLD = 30.0
rsi_results = []
all_rsi_values = {}

for ctype in CONSTANT_MODELS:
    rsi_vals = pti.momentum_indicators.bulk.relative_strength_index(
        close, constant_model_type=ctype, period=RSI_PERIOD)
    # Store for later visualization
    all_rsi_values[ctype] = rsi_vals
    start_idx = len(close) - len(rsi_vals)
    rating = 0
    total_signals = 0
    for offset, rsi in enumerate(rsi_vals[:-1]):  # skip last (no future bar)
        idx = start_idx + offset
        price = close[idx]
        if rsi < OVERSOLD:
            total_signals += 1
            next_price = close[idx + 1]
            if next_price > price:
                rating += 1
    success_rate = rating / total_signals if total_signals > 0 else 0.0
    rsi_results.append({
        "model": ctype,
        "period": RSI_PERIOD,
        "signals": total_signals,
        "correct_signals": rating,
        "success_rate": success_rate
    })

score_df = pd.DataFrame(rsi_results).sort_values("success_rate", ascending=False)
score_df

**Best Model:**

In [ ]:
best = score_df.iloc[0]
best_model = best['model']
print(f"Best model: {best_model} (Success Rate: {best['success_rate']:.2%})")

# Also add the best RSI as a column to the df for visualization
best_rsi = all_rsi_values[best_model]
df.loc[df.index[-len(best_rsi):], f"RSI_{RSI_PERIOD}_{best_model}"] = best_rsi

## 5. Visualize Price, SMA, and the Best RSI Variant

We'll overlay price, SMA, and the highest-scoring RSI variant (according to the previous evaluation).

In [ ]:
import plotly.graph_objs as go

rsi_col = f"RSI_{RSI_PERIOD}_{best_model}"

fig = go.Figure([
    go.Candlestick(
        x=df["Date"], open=df["Open"], high=df["High"], low=df["Low"], close=df["Close"], name="Price"),
    go.Scatter(x=df["Date"], y=df["SMA_20"], line=dict(color="orange"), name="SMA 20"),
    go.Scatter(x=df["Date"], y=df[rsi_col], yaxis="y2", name=f"RSI {RSI_PERIOD} {best_model}", line=dict(color="purple"))
])
fig.update_layout(
    title=f"BTCUSDT Price, SMA, and Best RSI ({best_model})",
    yaxis=dict(title="Price"),
    yaxis2=dict(title="RSI", overlaying="y", side="right"),
    template="plotly_white"
)
fig.show()

## 6. Extending Further

- Try different symbols (e.g. `ETHUSDT`, `BNBUSDT`)
- Change intervals (e.g. `4h`, `1h`)
- Add more indicator columns or custom logic for signals
- Save the enriched DataFrame to disk for further research

---

Happy systematic research! 🦀🐍📈